<a href="https://colab.research.google.com/github/sallywang147/SCInvarinfer/blob/main/llama_for_smart_contracts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
!pip install --quiet accelerate
!pip install -i --quiet https://test.pypi.org/simple/ bitsandbytes
!pip install --quiet sentencepiece
!pip install --quiet git+https://github.com/huggingface/transformers
!pip install --quiet rich[jupyter]
!pip install --quiet deepspeed
!pip install --quiet openai
!pip install --quiet gradio
!pip install --quiet peft
!pip install --quiet --upgrade datasets
!pip install --quiet simplejson
!pip install --quiet accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.3/215.3 KB 6.2 MB/s eta 0:00:00
Looking in indexes: --quiet, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.2/9.2 MB 62.6 MB/s eta 0:00:00
  ERROR: Cannot unpack file /tmp/pip-unpack-9qhtjozf/simple.html (downloaded from /tmp/pip-req-build-6on3dnsx, content-type: text/html); cannot detect archive format
ERROR: Cannot determine archive format of /tmp/pip-req-build-6on3dnsx
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 18.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 58.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 765.4/765.4 KB 12.9 MB/s

##Vanilla Llama v. 8bit_Quantized Llama##

vanilla llama is slightly more capable than quantized llama, but not by much. For example, vanilla llama tends to repeat the same information less. 

In [ ]:
import os
import numpy as np
import accelerate
import bitsandbytes
import transformers
import pandas as pd
import plotly.express as px
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler

from transformers import AutoTokenizer, LlamaTokenizer, LlamaForCausalLM

vanilla_model = LlamaForCausalLM.from_pretrained(
            "decapoda-research/llama-7b-hf",
            load_in_8bit=True,
            torch_dtype=torch.float16,
            device_map= "auto",
        )

vanilla_tokenizer = LlamaTokenizer.from_pretrained(
            "decapoda-research/llama-7b-hf",
        )
#model = LlamaForCausalLM.from_pretrained('sallywww/Llama-7B')
#tokenizer = AutoTokenizer.from_pretrained('sallywww/Llama-7B')

#model.push_to_hub('sallywww/Llama-7B')
#tokenizer.push_to_hub('sallywww/Llama-7B')


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching /usr/local/cuda/lib64...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.0
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.9/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: /usr/lib64-nvidia did not contain libcudart.so as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('--listen_host=172.28.0.12 --target_host=172.28.0.12 --tunnel_background_save_url=https'), PosixPath('//colab.research.google.com/tun/m/cc48301118ce562b961b3c22d803539adc1e0c19/gpu-a100-s-3o6r561ruj4qn --tunnel_background_save_delay=10s --tunnel_periodic_background_save_frequency=30m0s --enable_output_coalescing=true --output_coalescing_require

Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'LLaMATokenizer'. 
The class this function is called from is 'LlamaTokenizer'.


In [ ]:
'''
prompt = "can you tell me what's a loop invariant?" 
inputs = tokenizer(prompt, return_tensors="pt")

# Generate
generate_ids = model.generate(inputs.input_ids, max_length=100)
tokenizer.batch_decode(generate_ids, 
                       skip_special_tokens=True, 
                       temperature=0.8, 
                       top_p=0.95, 
                       top_k=50, 
                       repeat_penalty=0.2,
                       clean_up_tokenization_spaces=False)[0]
'''

'\nprompt = "can you tell me what\'s a loop invariant?" \ninputs = tokenizer(prompt, return_tensors="pt")\n\n# Generate\ngenerate_ids = model.generate(inputs.input_ids, max_length=100)\ntokenizer.batch_decode(generate_ids, \n                       skip_special_tokens=True, \n                       temperature=0.8, \n                       top_p=0.95, \n                       top_k=50, \n                       repeat_penalty=0.2,\n                       clean_up_tokenization_spaces=False)[0]\n'

##loading and json-ize data##

In [ ]:
from collections import OrderedDict
import simplejson as json
from numpy import random
import pandas as pd
import io
import os
import pandas
from google.colab import auth
from google.auth import default
from numpy import random
import pandas as pd
import gspread
import gc

#df = pandas.read_excel(open('/content/invariants_data.xlsx','rb'))
#df
#autenticating to google
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)
#defining my worksheet
worksheet = gc.open('invariants_line_number').sheet1
#get_all_values gives a list of rows
rows = worksheet.get_all_values()
#Convert to a DataFrame 
cols = ['Source', 'Target']
df = pd.DataFrame(rows, columns=cols)
df

,Source,Target
0,1 pragma solidity >=0.4.24 <0.6.0;\n\n2 contra...,14+ assert (y == x + 4);
1,1 // SPDX-License-Identifier: MIT\n2 pragma ex...,"109+ require(admin == msg.sender, ""Ownable: c..."
2,1 pragma solidity >=0.4.24 <0.6.0;\n\n2 contra...,7+ assert (y == x + 2);
3,1 pragma solidity >=0.4.24 <0.6.0;\n2 contract...,20+ assert(funcA2(funcA1())==12);\n20+ assert(...
4,1 pragma solidity >=0.4.24 <0.6.0;\n\n2 // Thi...,24+ assert(a == x + 1);\n32+ assert(a == x);\n...
...,...,...
98,1 pragma solidity >=0.4.24 <0.6.0;\n2 \n3 cont...,6+ assert (!false);\n6+ assert (!false);
99,1 // SPDX-License-Identifier: MIT\n2 pragma ...,6+ assert(fee + value != 0);\n6+ assert(fee ...
100,1 // SPDX-License-Identifier: MIT\n2 pragma ...,15+ assert(_required > 0); \n15+ assert(m_num...
101,1 / SPDX-License-Identifier: MIT\n2 pragma so...,6+ assert(address(this)==msg.sender);\n6+ as...


In [ ]:
def _make_w_io_base(f, mode: str):
    if not isinstance(f, io.IOBase):
        f_dirname = os.path.dirname(f)
        if f_dirname != "":
            os.makedirs(f_dirname, exist_ok=True)
        f = open(f, mode=mode)
    return f


def _make_r_io_base(f, mode: str):
    if not isinstance(f, io.IOBase):
        f = open(f, mode=mode)
    return f


def jdump(obj, f, mode="w", indent=4, default=str):
    """Dump a str or dictionary to a file in json format.
    Args:
        obj: An object to be written.
        f: A string path to the location on disk.
        mode: Mode for opening the file.
        indent: Indent for storing json dictionaries.
        default: A function to handle non-serializable entries; defaults to `str`.
    """
    f = _make_w_io_base(f, mode)
    if isinstance(obj, (dict, list)):
        json.dump(obj, f, indent=indent, default=default)
    elif isinstance(obj, str):
        f.write(obj)
    else:
        raise ValueError(f"Unexpected type: {type(obj)}")
    f.close()


def jload(f, mode="r"):
    """Load a .json file into a dictionary."""
    f = _make_r_io_base(f, mode)
    jdict = json.load(f)
    f.close()
    return jdict


In [ ]:
# List to hold dictionaries
def prepare_actor_model_data(df): 
    data_list=[]
    for index, row in df.iterrows():
        data = OrderedDict()
        data["user_input"] = row['Source']
        data["completion"] = row['Target']
        data_list.append(data)
    jdump(data_list, 'datasets/actor_training_data.json')

def prepare_rlhf_model_data(df): 
    data_list=[]
    for index, row in df.iterrows():
        data = OrderedDict()
        data["user_input"] = row['Source']
        data_list.append(data)
    jdump(data, '/datasets/reward_training_data.json')


def prepare_reward_model_data(df):
    data_list=[]    
    for index, row in df.iterrows():
        data = OrderedDict()
        data["user_input"] = row['Source']
        data["completion"] = row['Target']
        data["score"] = 5
        data_list.append(data)
    jdump(data, '/content/datasets/rlhf_training_data.json')


In [ ]:
prepare_actor_model_data(df)
prepare_rlhf_model_data(df)
prepare_reward_model_data(df)

In [ ]:
#from google.colab import files
#!zip -r /content/datasets.zip /content/datasets
#files.download("/content/datasets.zip")

In [ ]:
#from google.colab import files
#!zip -r /content/datasets.zip /content/datasets
#files.download("/content/datasets/train_data.txt")

In [ ]:
with open('/content/datasets/actor_training_data.json', "r") as f:
    data = json.load(f)
    with open('/content/datasets/train_data.txt', "a") as file:
      for d in data:
       # item = [str(d["user_input"]) + "\n" + str(d["completion"]) +"\n" + "\n" for d in data]
        file.write(str(d["user_input"]) + "\n" + str(d["completion"]) + "\n" + "<end of text>" + "\n")
    file.close()

##Finetuning with PEFT and 8bits## 

In [ ]:
!export CUDA_VISIBLE_DEVICES=0

In [ ]:
import os
import argparse
import random
import torch
#import gradio as gr
import torch.nn as nn
import bitsandbytes
from transformers import GenerationConfig
from datasets import Dataset
from peft import prepare_model_for_int8_training, LoraConfig, get_peft_model, PeftModel

model = None
tokenizer = None
eft_model = None

def maybe_load_models():
    global model
    global tokenizer
    model = LlamaForCausalLM.from_pretrained(
            "decapoda-research/llama-7b-hf",
            load_in_8bit=True,
            torch_dtype=torch.float16,
            device_map= "auto",
        )

    tokenizer = LlamaTokenizer.from_pretrained(
            "decapoda-research/llama-7b-hf",
        )
    #model = vanilla_model 
    #tokenzier = vanilla_tokenizer 

def reset_models():
    global model
    global tokenizer

    del model
    del tokenizer

    model = None
    tokenizer = None

def generate_text(
    model_name, 
    text, 
    temperature, 
    top_p, 
    top_k, 
    repeat_penalty,
    max_new_tokens,
):
    global model
    global tokenizer

    maybe_load_models()

    tokenizer.pad_token_id = 0

    if model_name and model_name != "None":
        model = PeftModel.from_pretrained(
            model, model_name,
            torch_dtype=torch.float16
        )

    inputs = vanilla_tokenizer(text, return_tensors="pt")
    input_ids = inputs["input_ids"].to(model.device)

    # llama_config = transformers.LlamaConfig()
    # print(llama_config)

    stopping_criteria_list = transformers.StoppingCriteriaList()
    generation_config = GenerationConfig(
        do_sample=True,
        temperature=temperature,  
        top_p=top_p,  
        top_k=top_k,  
        repeat_penalty=repeat_penalty,
        max_new_tokens=max_new_tokens,  
    )

    with torch.no_grad():
        generation_output = model.generate(
            input_ids=input_ids,
            attention_mask=torch.ones_like(input_ids),
            generation_config=generation_config,
            use_cache=True,
        )[0].cuda()

    output_text = tokenizer.decode(generation_output)
    return output_text.strip()

def tokenize_and_train(
    training_data,
    max_seq_length,
    micro_batch_size,
    gradient_accumulation_steps,
    epochs,
    learning_rate,
    lora_r,
    lora_alpha,
    lora_dropout,
    model_name):
    global model
    global tokenizer

    reset_models()
    maybe_load_models()
    tokenizer.pad_token_id = 0
    paragraphs = training_data.split("<end of text>")
    print("Number of samples: " + str(len(paragraphs)))
        
    def tokenize(item):
        result = tokenizer(
            item["text"],
            truncation=True,
            max_length=max_seq_length,
            padding="max_length",
        )
        return {
            "input_ids": result["input_ids"][:-1],
            "attention_mask": result["attention_mask"][:-1],
        }

    def to_dict(text):
        return {"text": text}

    paragraphs = [to_dict(x) for x in paragraphs]
    data = Dataset.from_list(paragraphs)
    data = data.shuffle().map(lambda x: tokenize(x))
    model = prepare_model_for_int8_training(model)

    model = get_peft_model(model, LoraConfig(
        r=lora_r,
        lora_alpha=lora_alpha,
        target_modules=["q_proj", "v_proj"],
        lora_dropout=lora_dropout,
        bias="none",
        task_type="CAUSAL_LM",
    ))
    output_dir = f"lora-{model_name}"

    print("Training...")

    training_args = transformers.TrainingArguments(
     
        per_device_train_batch_size=micro_batch_size, 
        gradient_accumulation_steps=gradient_accumulation_steps,  

        # Number of warmup steps for the learning rate scheduler. During these steps, 
        # the learning rate increases linearly from 0 to its initial value. Warmup helps
        #  to reduce the risk of very large gradients at the beginning of training, 
        # which could destabilize the model.
        # warmup_steps=100, 

        # The total number of training steps. The training process will end once this 
        # number is reached, even if not all the training epochs are completed.
        # max_steps=1500, 
        num_train_epochs=epochs,  
        learning_rate=learning_rate, 
        fp16=True,  
        logging_steps=20, 
        output_dir=output_dir, 
        save_total_limit=100,  
    )


    trainer = transformers.Trainer(
        # The pre-trained model that you want to fine-tune or train from scratch. 
        # 'model' should be an instance of a Hugging Face Transformer model, such as BERT, GPT-2, T5, etc.
        model=model, 

        # The dataset to be used for training. 'data' should be a PyTorch Dataset or 
        # a compatible format, containing the input samples and labels or masks (if required).
        train_dataset=data, 

        # The TrainingArguments instance created earlier, which contains various 
        # hyperparameters and configurations for the training process.
        args=training_args, 

        # A callable that takes a batch of samples and returns a batch of inputs for the model. 
        # This is used to prepare the input samples for training by batching, padding, and possibly masking.
        data_collator=transformers.DataCollatorForLanguageModeling( 
            tokenizer,  
            # Whether to use masked language modeling (MLM) during training. 
            # MLM is a training technique used in models like BERT, where some tokens in the 
            # input are replaced by a mask token, and the model tries to predict the 
            # original tokens. In this case, MLM is set to False, indicating that it will not be used.
            mlm=False, 
        ),
    )

    result = trainer.train(resume_from_checkpoint=False)

    model.save_pretrained(output_dir)
    
    reset_models()

    return result

 #lora hyperparameters from this paper: https://github.com/microsoft/LoRA/tree/main/examples/NLG      
if __name__ == "__main__":
    file = open('/content/datasets/train_data.txt', "r")
    training_file = file.read()
    tokenize_and_train(training_file,
    max_seq_length=4096,
    micro_batch_size=1,
    gradient_accumulation_steps=8,
    epochs=30,
    learning_rate=0.001,
    lora_r=8,
    lora_alpha=32,
    lora_dropout=0.1,
    model_name='llama7b') #pass on lora model name 

Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'LLaMATokenizer'. 
The class this function is called from is 'LlamaTokenizer'.


Number of samples: 104


Map:   0%|          | 0/104 [00:00<?, ? examples/s]

Training...


Step,Training Loss
20,1.139800
40,0.717500
60,0.516700
80,0.365300
100,0.280700
120,0.234300
140,0.195400
160,0.150900
180,0.129000
200,0.108100


In [ ]:
#from google.colab import files
#!zip -r /content/lora-llama7b.zip /content/lora-llama7b
#files.download("/content/lora-llama7b.zip")
!cp -r /content/lora-llama7b drive/MyDrive

##Inference with trained llama##

Let's make some inferences without targeted instruction finetuning yet

In [ ]:
import os

global lora_param
for d in os.listdir(): 
     if os.path.isdir(d) and d.startswith('lora-'):
              if '7b' in d and 'llama' in d: 
                print(d)
                lora_param = d
model_name ="/content/drive/MyDrive/lora-llama7b"

In [ ]:
import argparse
import random
import torch
#import gradio as gr
import torch.nn as nn
#import bitsandbytes
from datasets import Dataset
from peft import PeftConfig, LoraConfig, get_peft_model, PeftModel
from transformers import LlamaForCausalLM, LlamaTokenizer

def load_trained_llama(model_name):
  peft_model_id = model_name
  config = PeftConfig.from_pretrained(peft_model_id)
  model = LlamaForCausalLM.from_pretrained(config.base_model_name_or_path)
  model = PeftModel.from_pretrained(model, peft_model_id)
  tokenizer = LlamaTokenizer.from_pretrained(config.base_model_name_or_path)
  model.cuda()
  return model, tokenizer
def infer_invariants(model, tokenizer, test_data, max_new_tokens):
  model.eval()
  inputs = tokenizer(test_data, return_tensors="pt")
  with torch.no_grad():
      outputs = model.generate(input_ids=inputs["input_ids"].cuda(), \
      max_new_tokens=max_new_tokens)
      invariants = tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0]
  return invariants

In [ ]:
model, tokenizer = load_trained_llama(model_name)

Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'LLaMATokenizer'. 
The class this function is called from is 'LlamaTokenizer'.


In [ ]:
import sys
import io

contract = 'short_sushi_Trident.sol'
test_file = open(f'/content/drive/MyDrive/experiments/baseline benchmark/contextual_bugs_test/{contract}', "r", encoding="utf-8") 
test_data= test_file.read()

In [ ]:
output = infer_invariants(model=model, tokenizer=tokenizer, test_data=test_data, max_new_tokens=512)
print(output.strip())
with open(f"/content/drive/MyDrive/experiments/evaluation/quantized_llama_inferred_invariants/{contract}", "w", encoding="utf-8") as file:  
    file.write(output.strip()+"\n")
file.close()

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│ in infer_invariants:23                                                                           │
│                                                                                                  │
│ /usr/local/lib/python3.9/dist-packages/peft/peft_model.py:580 in generate                        │
│                                                                                                  │
│   577 │                                                                                          │
│   578 │   def generate(self, **kwargs):                                                          │
│   579 │   │   if not isinstance(self.peft_config, PromptLearningConfig):                         │
│ ❱ 580 │   │   │   return self.base_model.generate(**kwargs)                                      │
│   581 │   │   else:                                                                              │
│   582 │   │   │   if "input_ids" not in kwargs:                                                  │
│   583 │   │   │   │   raise ValueError("input_ids must be provided for Peft model generation")   │
│                                                                                                  │
│ /usr/local/lib/python3.9/dist-packages/torch/autograd/grad_mode.py:27 in decorate_context        │
│                                                                                                  │
│    24 │   │   @functools.wraps(func)                                                             │
│    25 │   │   def decorate_context(*args, **kwargs):                                             │
│    26 │   │   │   with self.clone():                                                             │
│ ❱  27 │   │   │   │   return func(*args, **kwargs)                                               │
│    28 │   │   return cast(F, decorate_context)                                                   │
│    29 │                                                                                          │
│    30 │   def _wrap_generator(self, func):                                                       │
│                                                                                                  │
│ /usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1416 in generate         │
│                                                                                                  │
│   1413 │   │   │   │   )                                                                         │
│   1414 │   │   │                                                                                 │
│   1415 │   │   │   # 11. run greedy search                                                       │
│ ❱ 1416 │   │   │   return self.greedy_search(                                                    │
│   1417 │   │   │   │   input_ids,                                                                │
│   1418 │   │   │   │   logits_processor=logits_processor,                                        │
│   1419 │   │   │   │   stopping_criteria=stopping_criteria,                                      │
│                                                                                                  │
│ /usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:2211 in greedy_search    │
│                                                                                                  │
│   2208 │   │   │   model_inputs = self.prepare_inputs_for_generation(input_ids, **model_kwargs)  │
│   2209 │   │   │                                                                                 │
│   2210 │   │   │   # forward pass to get next token                                              │
│ ❱ 2211 │   │   │   outputs = self(                         

In [ ]:
from peft import PeftConfig, prepare_model_for_int8_training, LoraConfig, get_peft_model, PeftModel
def generate_text(
    model_name, 
    text, 
    temperature, 
    top_p, 
    top_k, 
    repeat_penalty,
    max_new_tokens=350,
):

    vanilla_tokenizer.pad_token_id = 0
    model = prepare_model_for_int8_training(vanilla_model)
    model = PeftModel.from_pretrained(
            model, model_name,
            torch_dtype=torch.float16
        )
   # model.cuda()
    inputs = vanilla_tokenizer(text, return_tensors="pt")
    input_ids = inputs["input_ids"].cuda()
    stopping_criteria_list = transformers.StoppingCriteriaList()

    generation_config = GenerationConfig(
        do_sample=True,
        temperature=temperature,  
        top_p=top_p,  
        top_k=top_k,  
        repeat_penalty=repeat_penalty,
        max_new_tokens=max_new_tokens,  
    )


    with torch.no_grad():
        generation_output = model.generate(
            input_ids=input_ids,
            attention_mask=torch.ones_like(input_ids),
            generation_config=generation_config,
            use_cache=True,
        )[0].cuda()

    output_text = vanilla_tokenizer.decode(generation_output)
    return output_text.strip()

for d in os.listdir(): 
  if os.path.isdir(d) and d.startswith('lora-'):
      print(d)
      lora_model = d

inference = generate_text(
                model_name=lora_param,
                text=test_data,
                temperature=0.8,
                top_p=0.95,
                top_k=50,
                repeat_penalty=0.02,
                max_new_tokens=350)

lora-llama7b


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:49                                                                                   │
│ in generate_text:34                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.9/dist-packages/peft/peft_model.py:580 in generate                        │
│                                                                                                  │
│   577 │                                                                                          │
│   578 │   def generate(self, **kwargs):                                                          │
│   579 │   │   if not isinstance(self.peft_config, PromptLearningConfig):                         │
│ ❱ 580 │   │   │   return self.base_model.generate(**kwargs)                                      │
│   581 │   │   else:                                                                              │
│   582 │   │   │   if "input_ids" not in kwargs:                                                  │
│   583 │   │   │   │   raise ValueError("input_ids must be provided for Peft model generation")   │
│                                                                                                  │
│ /usr/local/lib/python3.9/dist-packages/torch/autograd/grad_mode.py:27 in decorate_context        │
│                                                                                                  │
│    24 │   │   @functools.wraps(func)                                                             │
│    25 │   │   def decorate_context(*args, **kwargs):                                             │
│    26 │   │   │   with self.clone():                                                             │
│ ❱  27 │   │   │   │   return func(*args, **kwargs)                                               │
│    28 │   │   return cast(F, decorate_context)                                                   │
│    29 │                                                                                          │
│    30 │   def _wrap_generator(self, func):                                                       │
│                                                                                                  │
│ /usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1462 in generate         │
│                                                                                                  │
│   1459 │   │   │   )                                                                             │
│   1460 │   │   │                                                                                 │
│   1461 │   │   │   # 13. run sample                                                              │
│ ❱ 1462 │   │   │   return self.sample(                                                           │
│   1463 │   │   │   │   input_ids,                                                                │
│   1464 │   │   │   │   logits_processor=logits_processor,                                        │
│   1465 │   │   │   │   logits_warper=logits_warper,                                              │
│                                                                                                  │
│ /usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:2478 in sample           │
│                                                                                                  │
│   2475 │   │   │   model_inputs = self.prepare_inputs_for_generation(input_ids, **model_kwargs)  │
│   2476 │   │   │                                                                                 │
│   2477 │   │   │   # forward pass to get next token                                              │
│ ❱ 2478 │   │   │   outputs = self(                         